In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
!pip install boruta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.4 MB/s eta 0:00:00


In [5]:
URL= "https://raw.githubusercontent.com/Aditya1001001/English-Premier-League/master/pos_modelling_data.csv"

In [6]:
df=pd.read_csv(URL)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1793 entries, 0 to 1792
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Position             1793 non-null   object 
 1   Clean sheets         1793 non-null   float64
 2   Goals conceded       1793 non-null   float64
 3   Tackles              1793 non-null   float64
 4   Tackle success %     1793 non-null   int64  
 5   Blocked shots        1793 non-null   float64
 6   Interceptions        1793 non-null   float64
 7   Clearances           1793 non-null   float64
 8   Recoveries           1793 non-null   float64
 9   Successful 50/50s    1793 non-null   float64
 10  Own goals            1793 non-null   float64
 11  Assists              1793 non-null   int64  
 12  Passes               1793 non-null   int64  
 13  Passes per match     1793 non-null   float64
 14  Big chances created  1793 non-null   float64
 15  Crosses              1793 non-null   f

In [8]:
X = df.drop('Position', axis=1)
y= df['Position']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)


In [9]:
rf_all_features= RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
rf_all_features.fit(X_train, y_train)
accuracy_score(y_test,rf_all_features.predict(X_test))

0.7298050139275766

In [10]:
rfc=RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector=BorutaPy(rfc,n_estimators='auto',verbose=2,random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	29
Tentative: 	5
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	29
Tentative: 	3
Rejected: 	2
Iteration: 	10 / 100
Confirmed: 	29
Tentative: 	3
Rejected: 	2
Iteration: 	11 / 100
Confirmed: 	29
Tentative: 	3
Rejected: 	2
Iteration: 	12 / 100
Confirmed: 	30
Tentative: 	2
Rejected: 	2
Iteration: 	13 / 100
Confirmed: 	30
Tentative: 	2
Rejected: 	2
Iteration: 	14 / 100
Confirmed: 	30
Tentative: 	2
Rejected: 	2
Iteration: 	15 / 100
Confirmed: 	30
Tentative: 	2
Rejected: 	2
Iteration: 	16 / 100
Confirmed: 	30
Tentative: 	2
Rejected: 	2
I

BorutaPy(estimator=RandomForestClassifier(max_depth=5, n_estimators=160,
                                          random_state=RandomState(MT19937) at 0x7B3473847740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7B3473847740, verbose=2)

In [11]:
print("Ranking: ", boruta_selector.ranking_)
print("No of Significant features: ", boruta_selector.n_features_)

Ranking:  [1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 2 1 1 1 1 1 1]
No of Significant features:  31


In [16]:
slected_rf_features = pd.DataFrame({'Features': list(X_train.columns), 'Ranking':boruta_selector.ranking_})
slected_rf_features.sort_values(by='Ranking')

,Features,Ranking
0,Clean sheets,1
31,Arial Saves,1
30,overall,1
29,value_eur,1
28,age,1
26,Saves,1
25,Shooting accuracy %,1
24,Shots,1
23,Goals per match,1
22,Goals,1


In [17]:
X_imp_train = boruta_selector.transform(np.array(X_train))
X_imp_test=boruta_selector.transform(np.array(X_test))

In [18]:
rf_boruta= RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
rf_boruta.fit(X_imp_train, y_train)


RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=1)

In [19]:
accuracy_score(y_test,rf_boruta.predict(X_imp_test))

0.7325905292479109